In [1]:
import yfinance as yf
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

# 1. ACCIONES

In [69]:
tickers2 = ['WALMEX.MX','ELEKTRA.MX','BIMBOA.MX','GAPB.MX','CEMEXCPO.MX']
tickers3 = ['AMXB.MX','CEMEXCPO.MX','FEMSAUBD.MX','BBAJIOO.MX','WALMEX.MX','ALFAA.MX','ELEKTRA.MX','BIMBOA.MX','GFNORTEO.MX','AC.MX',"CUERVO.MX",'GAPB.MX','ASURB.MX','NAFTRACISHRS.MX']

In [70]:
tickers = pd.read_excel('infoDownload.xlsx')
tickers = tickers["Símbolo"].tolist()
no = ["PEOLES.MX", "MEXCHEM.MX", "LIVEPOLC1.MX", "KOFL.MX", "SITESB1.MX",'ALPEKA.MX']

for i in no:
    tickers.remove(i)
tickers2

['WALMEX.MX', 'ELEKTRA.MX', 'BIMBOA.MX', 'GAPB.MX', 'CEMEXCPO.MX']

In [71]:
# Bajamos los ETF de yahoo
start_date = '2023-03-01'

St = yf.download(tickers2, start=start_date)['Close']
St.head()
rt = St.pct_change().dropna()
rt

[*********************100%***********************]  5 of 5 completed


,BIMBOA.MX,CEMEXCPO.MX,ELEKTRA.MX,GAPB.MX,WALMEX.MX
Date,,,,,
2023-03-02,0.011257,0.042598,0.005508,0.002711,0.001687
2023-03-03,0.004884,0.038815,0.010664,0.007907,0.016000
2023-03-06,-0.006556,-0.020649,-0.017002,-0.019410,0.008703
2023-03-07,-0.020822,-0.034137,-0.007338,-0.007059,-0.006984
2023-03-08,0.004997,0.005198,0.004784,0.002903,-0.000138
2023-03-09,-0.003931,-0.010341,-0.001608,-0.010692,-0.006345
2023-03-13,-0.009170,-0.010449,0.000000,0.028242,0.027485
2023-03-14,-0.002812,-0.004224,0.001874,-0.009088,-0.004458
2023-03-15,0.005522,-0.021209,-0.012377,-0.018137,0.007328


In [72]:
plazo = 21

resumen = pd.DataFrame()
resumen['media'] = rt.mean()*plazo
resumen['vol'] = rt.std()*np.sqrt(plazo)

max_res = resumen[resumen.media == resumen.media.max()]
resumen

,media,vol
BIMBOA.MX,0.035445,0.053199
CEMEXCPO.MX,0.072475,0.090933
ELEKTRA.MX,0.131133,0.075739
GAPB.MX,0.020475,0.062577
WALMEX.MX,0.002383,0.055310


In [73]:
# Matriz de covarianza
sigma = rt.cov()

In [74]:
# Tasa libre de riesgo anual
anual = 0.1152
mensual = anual/12
diaria = anual/252
rf = mensual

In [75]:
# Definimos el número de portafolios que simularemos, y la cantidad de activos que tenemos
n_port = 1000000
n_act = len(rt.keys())

In [76]:
W = np.random.dirichlet((1,) * n_act, size=n_port)

In [77]:
# Rendimientos y volatilidad de cada portafolios
Valor_esperado = W.dot(resumen['media'])

volatilidad = [] 

for i in range(len(W)):
    varianza = W[i].T.dot(sigma).dot(W[i])
    vol = np.sqrt(varianza)    
    volatilidad.append(vol * np.sqrt(plazo))


In [78]:
# Radio de Sharpe
rs = (Valor_esperado - rf)/ volatilidad

In [79]:
# Data frame de resultados
resultados = pd.DataFrame(W)
resultados.columns = tickers2
resultados['rendimientos'] = Valor_esperado
resultados['vol'] = volatilidad
resultados['sharpe'] = rs

In [80]:
max_sharpe = resultados[resultados.sharpe == resultados.sharpe.max()]
max_sharpe

,WALMEX.MX,ELEKTRA.MX,BIMBOA.MX,GAPB.MX,CEMEXCPO.MX,rendimientos,vol,sharpe
895400,0.000801,0.109085,0.720809,0.001519,0.167785,0.102887,0.056517,1.650589


In [81]:
max_rend = resultados[resultados.rendimientos == resultados.rendimientos.max()]
max_rend

,WALMEX.MX,ELEKTRA.MX,BIMBOA.MX,GAPB.MX,CEMEXCPO.MX,rendimientos,vol,sharpe
266461,0.010164,0.007528,0.974409,0.004094,0.003805,0.128776,0.074196,1.606232


In [82]:
min_vol = resultados[resultados.vol == resultados.vol.min()]
min_vol

,WALMEX.MX,ELEKTRA.MX,BIMBOA.MX,GAPB.MX,CEMEXCPO.MX,rendimientos,vol,sharpe
477838,0.355396,0.000153,0.152416,0.049244,0.442791,0.034658,0.031241,0.8021


In [94]:
med_vol = resultados.query('vol < 0.055')
med_vol = med_vol[med_vol.rendimientos == med_vol.rendimientos.max()]
med_vol

,WALMEX.MX,ELEKTRA.MX,BIMBOA.MX,GAPB.MX,CEMEXCPO.MX,rendimientos,vol,sharpe
908339,0.0254,0.044315,0.725714,0.002438,0.202133,0.099808,0.054927,1.642346


In [95]:
max_sharpe = resultados[resultados.sharpe == resultados.sharpe.max()]
no = ['rendimientos','vol','sharpe']
max_sharpe2 = max_sharpe.drop(no, axis=1)

med_vol2 = med_vol.drop(no, axis=1)


In [96]:
ult_precios = St.iloc[-1]
ult_precios

BIMBOA.MX        90.809998
CEMEXCPO.MX      10.220000
ELEKTRA.MX     1119.020020
GAPB.MX         351.070007
WALMEX.MX        71.209999
Name: 2023-04-12 00:00:00, dtype: float64

In [103]:
capital = 1000000

In [104]:
#max sharpe

emv_max = max_sharpe.rendimientos.values[0]
s_emv_max = max_sharpe.vol.values[0]
np.round((emv_max,s_emv_max),4)

array([0.1029, 0.0565])

In [105]:
#max sharpe

sharpeT = max_sharpe2.T
sharpeT.index.name = 'Accion'
sharpeT = max_sharpe2.T.sort_values('Accion')

sharpeT.columns = ['Pesos']
sharpeT['Capital'] = sharpeT['Pesos']*capital
sharpeT['Contratos'] = (sharpeT['Capital']/ult_precios).apply(np.floor)

display(np.round(sharpeT,3),"Rendimiento: "+str(np.round(emv_max,3)),"Volatilidad: " +str(np.round(s_emv_max,3)), "Contratos: "+str(np.round(sharpeT['Contratos'].sum(),3)))

,Pesos,Capital,Contratos
Accion,,,
BIMBOA.MX,0.721,720809.358,7937.0
CEMEXCPO.MX,0.168,167784.907,16417.0
ELEKTRA.MX,0.109,109085.117,97.0
GAPB.MX,0.002,1519.321,4.0
WALMEX.MX,0.001,801.297,11.0


'Rendimiento: 0.103'

'Volatilidad: 0.057'

'Contratos: 24466.0'

In [106]:
# Med Vol

emv = med_vol.rendimientos.values[0]
s_emv = med_vol.vol.values[0]
np.round((emv,s_emv),4)

array([0.0998, 0.0549])

In [107]:
# Med Vol

medT = med_vol2.T
medT.index.name = 'Accion'
medT = med_vol2.T.sort_values('Accion')

medT.columns = ['Pesos']
medT['Capital'] = medT['Pesos']*capital
medT['Contratos'] = (medT['Capital']/ult_precios).apply(np.floor)

display(np.round(medT,3),"Rendimiento: "+str(np.round(emv,3)),"Volatilidad: " +str(np.round(s_emv,3)), "Contratos: "+str(np.round(medT['Contratos'].sum(),3)))

,Pesos,Capital,Contratos
Accion,,,
BIMBOA.MX,0.726,725713.904,7991.0
CEMEXCPO.MX,0.202,202133.207,19778.0
ELEKTRA.MX,0.044,44314.511,39.0
GAPB.MX,0.002,2438.153,6.0
WALMEX.MX,0.025,25400.225,356.0


'Rendimiento: 0.1'

'Volatilidad: 0.055'

'Contratos: 28170.0'

In [108]:
# pesos iguales

sharpeT['Pesos'] = 1/len(max_sharpe2.T)
sharpeT['Capital'] = sharpeT['Pesos']*capital
sharpeT['Contratos'] = (sharpeT['Capital']/ult_precios).apply(np.floor)

val_esperado_pesos_iguales = sharpeT['Pesos'].dot(resumen['media'])

varianza = sharpeT['Pesos'].T.dot(sigma).dot(sharpeT['Pesos'])
vol = np.sqrt(varianza)    
volatilidad_pesos_iguales = (vol * np.sqrt(plazo))

display(np.round(sharpeT,3),"Rendimiento: "+str(np.round(val_esperado_pesos_iguales,3)),"Volatilidad: " +str(np.round(volatilidad_pesos_iguales,3)))

,Pesos,Capital,Contratos
Accion,,,
BIMBOA.MX,0.2,200000.0,2202.0
CEMEXCPO.MX,0.2,200000.0,19569.0
ELEKTRA.MX,0.2,200000.0,178.0
GAPB.MX,0.2,200000.0,569.0
WALMEX.MX,0.2,200000.0,2808.0


'Rendimiento: 0.052'

'Volatilidad: 0.04'

# 2. BONOS

In [109]:
bonos = ["BLK1MASB2-A.MX","BLKBONOB2-A.MX","BLKCORB1-B.MX","BLKDINB0-D.MX","BLKFIHB1-C.MX","BLKGLO2B1-A.MX", "BLKGUB1B0-C.MX","BLKLIQ+B0-B.MX","BLKPZOB2-A.MX","BLKREALB2-A.MX","BLKUDI+B2-A.MX", "BLKUSLQB0-D.MX","GOLD1+B1-C.MX"]

start_date = '2015-01-01'

Stb = yf.download(bonos, start=start_date)['Close']
Stb.head()
rtb = Stb.pct_change().dropna()


[*********************100%***********************]  13 of 13 completed


In [ ]:
plazo = 252

resumenb = pd.DataFrame()
resumenb['media'] = rtb.mean()*plazo
resumenb['vol'] = rtb.std()*np.sqrt(plazo)
resumenb

In [ ]:
n_port = 1000000
n_actb = len(rtb.keys())

In [ ]:
W = np.random.dirichlet((1,) * n_actb, size=n_port)

In [ ]:
sigmab = rtb.cov()

In [ ]:
Valor_esperado = W.dot(resumenb['media'])
Valor_esperado.max()

volatilidad = [] 

for i in range(len(W)):
    varianza = W[i].T.dot(sigmab).dot(W[i])
    vol = np.sqrt(varianza)    
    volatilidad.append(vol * np.sqrt(plazo))

rs = (Valor_esperado - rf)/ volatilidad

resultadosb = pd.DataFrame(W)
resultadosb.columns = bonos
resultadosb['rendimientos'] = Valor_esperado
resultadosb['vol'] = volatilidad
resultadosb['sharpe'] = rs

In [ ]:
max_sharpeb = resultadosb[resultadosb.sharpe == resultadosb.sharpe.max()]
max_sharpeb

In [ ]:
max_renb = resultadosb[resultadosb.rendimientos == resultadosb.rendimientos.max()]
max_renb

In [ ]:
min_volb = resultadosb[resultadosb.vol == resultadosb.vol.min()]
min_volb

In [ ]:
med_volb = resultadosb.query('vol < 0.004')
med_volb = med_volb[med_volb.rendimientos == med_volb.rendimientos.max()]
med_volb

In [ ]:
max_renb = resultadosb[resultadosb.rendimientos == resultadosb.rendimientos.max()]
no = ['rendimientos','vol','sharpe']
max_ren2b = max_renb.drop(no, axis=1)
np.round(max_ren2b,3)

In [ ]:
med_vol2b = med_volb.drop(no, axis=1)
np.round(med_vol2b,3)

In [ ]:
ult_preciosb = Stb.iloc[-1]
ult_preciosb

In [ ]:
capitalb = 2000000-capital

#max ren
renTb = max_ren2b.T
renTb.index.name = 'Bono'
renTb = max_ren2b.T.sort_values('Bono')

renTb.columns = ['Pesos']
renTb['Capital'] = renTb['Pesos']*capitalb
renTb['Contratos'] = (renTb['Capital']/ult_preciosb).apply(np.floor)


np.round(renTb,3)

In [ ]:
#MED VOL
medTb = med_vol2b.T
medTb.index.name = 'Bono'
medTb = med_vol2b.T.sort_values('Bono')

medTb.columns = ['Pesos']
medTb['Capital'] = medTb['Pesos']*capitalb
medTb['Contratos'] = (medTb['Capital']/ult_preciosb).apply(np.floor)

np.round(medTb,3)

In [ ]:
#max ren
ren_emvb = max_renb.rendimientos.values[0]
ren_s_emvb = max_renb.vol.values[0]
np.round((ren_emvb,ren_s_emvb),3)

In [ ]:
# med vol
med_emvb = med_volb.rendimientos.values[0]
med_s_emvb = med_volb.vol.values[0]
np.round((med_emvb,med_s_emvb),3)

In [ ]:
ren_utilidadb = (capitalb *(1+ren_emvb))-capitalb
ren_utilidadb

In [ ]:
med_utilidadb = (capitalb *(1+med_emvb))-capitalb
med_utilidadb

# 3. BACKTESTING


In [ ]:
tickers = ['AAPL','AMZN','MSFT','META','TSLA', 'SBUX', 'COST', 'WMT', 'VOO','EWW','QQQ','SPYD','FEMS']

In [ ]:
start_date = '2015-01-01'

St = yf.download(tickers, start=start_date, end = '2020-04-01')['Close']
St.head()
rt = St.pct_change().dropna()

In [ ]:
plazo = 252

resumen = pd.DataFrame()
resumen['media'] = rt.mean()*plazo
resumen['vol'] = rt.std()*np.sqrt(plazo)

max_res = resumen[resumen.media == resumen.media.max()]
resumen

In [ ]:
sigma = rt.cov()

In [ ]:
anual = 0.1152
mensual = anual/12
diaria = anual/252
rf = anual

In [ ]:
n_port = 1000000
n_act = len(rt.keys())
W = np.random.dirichlet((1,) * n_act, size=n_port)

In [ ]:
Valor_esperado = W.dot(resumen['media'])

volatilidad = [] 

for i in range(len(W)):
    varianza = W[i].T.dot(sigma).dot(W[i])
    vol = np.sqrt(varianza)    
    volatilidad.append(vol * np.sqrt(plazo))


In [ ]:
rs = (Valor_esperado - rf)/ volatilidad

In [ ]:
resultados = pd.DataFrame(W)
resultados.columns = tickers
resultados['rendimientos'] = Valor_esperado
resultados['vol'] = volatilidad
resultados['sharpe'] = rs

In [ ]:
max_sharpe = resultados[resultados.sharpe == resultados.sharpe.max()]
max_sharpe

In [ ]:
max_sharpe = resultados[resultados.sharpe == resultados.sharpe.max()]
no = ['rendimientos','vol','sharpe']
max_sharpe2 = max_sharpe.drop(no, axis=1)

In [ ]:
emv_max = max_sharpe.rendimientos.values[0]
s_emv_max = max_sharpe.vol.values[0]
np.round((emv_max,s_emv_max),4)

In [ ]:
precios_iniciales = St.loc['2020-03-31']
precios_iniciales

In [ ]:
start_date = '2015-01-01'

St2 = yf.download(tickers, start=start_date)['Close']
St2.head()
rt2 = St2.pct_change().dropna()

In [ ]:
ult_precios = St2.iloc[-1]
ult_precios

In [ ]:
sharpeT = max_sharpe2.T
sharpeT.index.name = 'Accion'
sharpeT = max_sharpe2.T.sort_values('Accion')

sharpeT.columns = ['Pesos']
#sharpeT['Pesos'] = 1/len(max_sharpe2.T)
sharpeT['Capital'] = sharpeT['Pesos']*capital
sharpeT['%Capital'] = sharpeT['Capital']/capital
sharpeT['$inicial'] = precios_iniciales
sharpeT['Contratos'] = sharpeT['Capital']/precios_iniciales
sharpeT['$final'] = ult_precios
sharpeT['Posicion'] = ult_precios*sharpeT['Contratos']
sharpeT['Utilidad'] = ult_precios*sharpeT['Contratos']-sharpeT['Capital']
sharpeT['%'] = (sharpeT['Posicion']/sharpeT['Capital'])-1

display(np.round(sharpeT,3),"Rendimiento: "+str(np.round(emv_max,3)),"Volatilidad: " +str(np.round(s_emv_max,3)), "Contratos: "+str(np.round(sharpeT['Contratos'].sum(),3)))

In [ ]:
ut = sharpeT['Utilidad'].sum()
ut

In [ ]:
ut_backtesting = ((ut + capital )/capital)-1
ut_backtesting

In [ ]:
plt.plot(St)

In [ ]:
plt.plot(St2)